<a href="https://colab.research.google.com/github/Ayanlola2002/NLP-for-Business/blob/main/NLP_BP_PROJECT2_Emmanuel_Ayanlowo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Run in terminal or command prompt
!python3 -m spacy download en

     |████████████████████████████████| 12.0 MB 5.4 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [4]:
!pip install pyLDAvis

     |████████████████████████████████| 1.7 MB 5.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=8de511b848702a89640dafc66e7321d63fd1c15cdd086381957d1b78b63454ae
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis


In [6]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

import gensim
from nltk.corpus import stopwords
import nltk; 
nltk.download('stopwords')


# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel


# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
import urllib
url = "https://www.gutenberg.org/files/67278/67278-0.txt"
file = urllib.request.urlopen(url)
data=[]
for line in file:
  decoded_line = line.decode("utf-8")
  data.append(decoded_line)
 

In [8]:
data

['\ufeffThe Project Gutenberg eBook of A Secret Service: Being Strange Tales\r\n',
 'of a Nihilist, by William Le Queux\r\n',
 '\r\n',
 'This eBook is for the use of anyone anywhere in the United States and\r\n',
 'most other parts of the world at no cost and with almost no restrictions\r\n',
 'whatsoever. You may copy it, give it away or re-use it under the terms\r\n',
 'of the Project Gutenberg License included with this eBook or online at\r\n',
 'www.gutenberg.org. If you are not located in the United States, you\r\n',
 'will have to check the laws of the country where you are located before\r\n',
 'using this eBook.\r\n',
 '\r\n',
 'Title: A Secret Service: Being Strange Tales of a Nihilist\r\n',
 '\r\n',
 'Author: William Le Queux\r\n',
 '\r\n',
 'Release Date: January 29, 2022 [eBook #67278]\r\n',
 '\r\n',
 'Language: English\r\n',
 '\r\n',
 'Produced by: Carlos Colon, the University of California and the Online\r\n',
 '             Distributed Proofreading Team at https://www.pg

In [9]:
print(data[2])
print('\n\n')
print(data[100])
print('\n\n')
print(data[300])






shivering humanity trudging through trackless snows and driven to their




military service until the day arrived when I was free to return and seek



In [10]:
# pre-process the text data
#removing emails and unwanted xters text 
def remove_emails_newlinexters(dt):
  #remove emails
  dt = [re.sub('\S*@\S*\s?', '', sent) for sent in dt]
  # Remove new line characters
  dt = [re.sub('\s+', ' ', sent) for sent in dt]
  # Remove distracting single quotes
  dt = [re.sub("\'", "", sent) for sent in dt]
  return dt

#tokenization
def sent_to_words(sentences):
  for sentence in sentences:
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
#stop-words removal
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def remove_stopwords(texts):
  return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

#lemmatization
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out


In [11]:
#Function call to remove emails and unwanted xter
data=remove_emails_newlinexters(data)

In [12]:
#tokenisation words
data_words = list(sent_to_words(data))

print(data_words[:1])

[['the', 'project', 'gutenberg', 'ebook', 'of', 'secret', 'service', 'being', 'strange', 'tales']]


In [13]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

In [14]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

In [15]:
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_nostops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])


In [16]:
print(data_lemmatized[:1])

[['project', 'secret', 'service', 'strange', 'tale']]


In [17]:
# import packages from scikit-learn
from sklearn.decomposition import LatentDirichletAllocation, NMF, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [20]:
vectorizer = CountVectorizer( 
                             stop_words='english', lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(data_lemmatized[0])

In [22]:
# Build a Latent Dirichlet Allocation Model
lda_model = LatentDirichletAllocation(n_components= 10, max_iter=10, learning_method='online') # no of components = no. of topics
lda_Z = lda_model.fit_transform(data_vectorized)
print(lda_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
# Build a Non-Negative Matrix Factorization Model
nmf_model = NMF(n_components= 10)
nmf_Z = nmf_model.fit_transform(data_vectorized)
print(nmf_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
# Build a Latent Semantic Indexing Model
lsi_model = TruncatedSVD(n_components=4)
lsi_Z = lsi_model.fit_transform(data_vectorized)
print(lsi_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)

(5, 10)
(5, 10)
(5, 4)


/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:294: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  FutureWarning,


In [23]:
# inspect the inferred topics
def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])

In [24]:
print("LDA Model:")
print_topics(lda_model, vectorizer)
print("=" * 20)

LDA Model:
Topic 0:
[('service', 0.286438980871638), ('project', 0.2815397584398642), ('secret', 0.2728890751478443), ('tale', 0.2600591415782216), ('strange', 0.2557984896471765)]
Topic 1:
[('project', 1.118692235271339), ('tale', 0.2755903668017779), ('strange', 0.2653541455527318), ('secret', 0.2553324740337015), ('service', 0.23118865278638)]
Topic 2:
[('secret', 1.0986711656200403), ('service', 0.2847793354624922), ('project', 0.28321897625109854), ('tale', 0.27303517263045135), ('strange', 0.24713910443066028)]
Topic 3:
[('service', 1.1118004362594198), ('strange', 1.0881421885214213), ('secret', 0.27966958862735924), ('project', 0.24935687112491153), ('tale', 0.24910145691414992)]
Topic 4:
[('secret', 0.2846308655960114), ('service', 0.28279373764980387), ('project', 0.26112971047398925), ('tale', 0.25144042715451687), ('strange', 0.24019307848549395)]
Topic 5:
[('service', 0.28068254310735075), ('strange', 0.2646169146427338), ('project', 0.2617016133812142), ('secret', 0.25440

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [25]:
print("NMF Model:")
print_topics(nmf_model, vectorizer)
print("=" * 20)

NMF Model:
Topic 0:
[('strange', 0.16153351274954492), ('tale', 0.0), ('service', 0.0), ('secret', 0.0), ('project', 0.0)]
Topic 1:
[('project', 0.1382817549901294), ('tale', 0.0), ('strange', 0.0), ('service', 0.0), ('secret', 0.0)]
Topic 2:
[('strange', 2.1500192957024478), ('tale', 0.0), ('service', 0.0), ('secret', 0.0), ('project', 0.0)]
Topic 3:
[('tale', 0.2849331398414937), ('strange', 0.0), ('service', 0.0), ('secret', 0.0), ('project', 0.0)]
Topic 4:
[('strange', 0.7268590442921552), ('tale', 0.0), ('service', 0.0), ('secret', 0.0), ('project', 0.0)]
Topic 5:
[('project', 0.8420400685422509), ('tale', 0.0), ('strange', 0.0), ('service', 0.0), ('secret', 0.0)]
Topic 6:
[('secret', 1.014075596395444), ('tale', 0.0), ('strange', 0.0), ('service', 0.0), ('project', 0.0)]
Topic 7:
[('service', 0.5955777636083325), ('tale', 0.0), ('strange', 0.0), ('secret', 0.0), ('project', 0.0)]
Topic 8:
[('strange', 0.21143619764721408), ('tale', 0.0), ('service', 0.0), ('secret', 0.0), ('proje

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [26]:
print("LSI Model:")
print_topics(lsi_model, vectorizer)
print("=" * 20)

LSI Model:
Topic 0:
[('project', 1.0), ('tale', 0.0), ('strange', 0.0), ('service', 0.0), ('secret', 0.0)]
Topic 1:
[('tale', 0.8549645998549158), ('secret', 0.4274822999274577), ('strange', 0.28498819995163854), ('service', 0.07124704998790964), ('project', -0.0)]
Topic 2:
[('strange', 0.8241751635132218), ('secret', 0.27713178652433085), ('service', 0.2374387186422668), ('project', -0.0), ('tale', -0.43307750765342823)]
Topic 3:
[('service', 0.8370822459474733), ('secret', 0.34442625477604194), ('project', 0.0), ('tale', -0.1046494471218346), ('strange', -0.4119616022854274)]


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [27]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:827: FutureWarning: 'square_distan

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
3      32.420952   8.115230       1        1  24.999414
2       0.648955  71.707100       2        1  14.999877
7      71.019684   9.971006       3        1  14.999858
1       3.785981  33.185135       4        1  14.999853
9     -42.666500  -9.222070       5        1   5.000192
8      -3.621689 -43.526516       6        1   5.000173
0      -3.632402  -4.171708       7        1   5.000164
6     -36.584400  35.804413       8        1   5.000162
5      44.391380  50.615383       9        1   5.000154
4      40.235809 -31.591665      10        1   5.000154, topic_info=      Term      Freq     Total Category  logprob  loglift
0  project  0.000000  0.000000  Default   5.0000   5.0000
4     tale  0.000000  0.000000  Default   4.0000   4.0000
1   secret  0.000000  0.000000  Default   3.0000   3.0000
2  service  1.000000  1.000000  Default   2.0000   2.0000
3  strange  1.000000  1.000000  Default   1.0000   1.0000
3  strange  0.456720  1.009381   Topic1  -1.0068   0.5933
2  service  0.466650  1.054299   Topic1  -0.9853   0.5713
1   secret  0.117384  0.976286   Topic1  -2.3654  -0.7320
4     tale  0.104554  0.977268   Topic1  -2.4812  -0.8487
0  project  0.104661  0.982766   Topic1  -2.4801  -0.8533
1   secret  0.376797  0.976286   Topic2  -0.6884   0.9451
0  project  0.097132  0.982766   Topic2  -2.0440  -0.4172
4     tale  0.093639  0.977268   Topic2  -2.0806  -0.4482
2  service  0.097667  1.054299   Topic2  -2.0385  -0.4819
3  strange  0.084758  1.009381   Topic2  -2.1803  -0.5802
4     tale  0.381414  0.977268   Topic3  -0.6762   0.9563
1   secret  0.099676  0.976286   Topic3  -2.0181  -0.3847
2  service  0.098743  1.054299   Topic3  -2.0275  -0.4710
0  project  0.084188  0.982766   Topic3  -2.1870  -0.5602
3  strange  0.085973  1.009381   Topic3  -2.1660  -0.5659
0  project  0.390936  0.982766   Topic4  -0.6515   0.9753
4     tale  0.096307  0.977268   Topic4  -2.0525  -0.4201
3  strange  0.092730  1.009381   Topic4  -2.0904  -0.4903
1   secret  0.089228  0.976286   Topic4  -2.1289  -0.4954
2  service  0.080791  1.054299   Topic4  -2.2282  -0.6716
0  project  0.053682  0.982766   Topic5  -1.5384   0.0884
4     tale  0.051470  0.977268   Topic5  -1.5805   0.0519
2  service  0.050783  1.054299   Topic5  -1.5939  -0.0374
1   secret  0.046623  0.976286   Topic5  -1.6794  -0.0460
3  strange  0.047451  1.009381   Topic5  -1.6618  -0.0617
0  project  0.052139  0.982766   Topic6  -1.5676   0.0592
4     tale  0.050470  0.977268   Topic6  -1.6001   0.0323
2  service  0.051736  1.054299   Topic6  -1.5754  -0.0188
1   secret  0.047598  0.976286   Topic6  -1.6587  -0.0253
3  strange  0.048066  1.009381   Topic6  -1.6489  -0.0488
0  project  0.051880  0.982766   Topic7  -1.5726   0.0543
1   secret  0.050286  0.976286   Topic7  -1.6038   0.0297
2  service  0.052783  1.054299   Topic7  -1.5553   0.0013
4     tale  0.047922  0.977268   Topic7  -1.6519  -0.0195
3  strange  0.047137  1.009381   Topic7  -1.6684  -0.0683
4     tale  0.056076  0.977268   Topic8  -1.4948   0.1377
3  strange  0.050647  1.009381   Topic8  -1.5966   0.0035
0  project  0.048839  0.982766   Topic8  -1.6330  -0.0061
1   secret  0.046325  0.976286   Topic8  -1.6858  -0.0524
2  service  0.048120  1.054299   Topic8  -1.6478  -0.0912
0  project  0.049857  0.982766   Topic9  -1.6123   0.0145
2  service  0.053473  1.054299   Topic9  -1.5423   0.0142
3  strange  0.050412  1.009381   Topic9  -1.6013  -0.0012
1   secret  0.048467  0.976286   Topic9  -1.6406  -0.0072
4     tale  0.047799  0.977268   Topic9  -1.6545  -0.0220
1   secret  0.053901  0.976286  Topic10  -1.5343   0.0991
2  service  0.053553  1.054299  Topic10  -1.5408   0.0157
0  project  0.049451  0.982766  Topic10  -1.6205   0.0063
4     tale  0.047616  0.977268  Topic10  -1.6583  -0.0259
3  strange  0.045486  1.009381  Topic10  -1.7041  -0.1040, token_table=Empty DataFrame
Columns: